In [4]:
# !pip install linearmodels # uncomment if linear models is not installed
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from scipy.optimize import fsolve
from IPython.display import display, Markdown

In [7]:
class berry_ml:
    def __init__(self,
                n = 100, # number of montecarlo simulations 
                m=500, # number of markets
                j = 2, # number of firms
                beta0 = 5, # utility parameter constant
                betax = 2, # utility parameter for observed characters
                alpha = 1, # utility parameter for price
                gamma0 = 1, # parameter for cost (intercept)
                gammax = 0.5, # parameter for cost (observed characters)
                gammaw = 0.25, # parameter for cost
                sigmad = 1, # utility parameter (unobservable characters)
                sigmac = 0.25, # cost parameter (unobservable characters)
                sigmaomega = 0.25): # cost parameter (other unobservables)
                      
        # Save parameters
        self.m, self.n, self.j, self.beta0, self.betax, self.alpha, self.gamma0= m,n,j,beta0,betax,alpha,gamma0
        self.gammax, self.gammaw, self.sigmad, self.sigmac, self.sigmaomega = gammax, gammaw, sigmad, sigmac, sigmaomega
    
    # main function of class berry - using functions defined below, it run the simulations and gives output
    def simulations(self):
        np.random.seed(68309)
        results = pd.DataFrame(np.zeros((self.n, 6)))
        for i in range(self.n):
            results.iloc[i] = data_reg(self.m, self.j, self.gamma0, self.gammax, self.sigmac, self.gammaw, self.sigmaomega, self.beta0, self.betax, self.sigmad, self.alpha)

        # take mean of estimates of all simulations:
        final_means = results.mean().round(3)
        final_sd = results.std().round(3)

        # Restructure everything to print as Markdown
        values = [
            [f"${self.beta0}$", f"${final_means.iloc[0]}$ <br> ({final_sd.iloc[0]})", f"${final_means.iloc[3]}$ <br> ({final_sd.iloc[3]})"],  # beta_0 row
            [f"${self.betax}$", f"${final_means.iloc[1]}$ <br> ({final_sd.iloc[1]})", f"${final_means.iloc[4]}$ <br> ({final_sd.iloc[4]})"],  # beta_x row
            [f"-${self.alpha}$", f"${final_means.iloc[2]}$ <br> ({final_sd.iloc[2]})", f"${final_means.iloc[5]}$ <br> ({final_sd.iloc[5]})"]   # alpha row
        ]

        # Define row and column names
        row_names = ['$\\beta_0$', '$\\beta_x$', '$\\alpha$']
        column_names = ['True Value', 'OLS', 'IV']
    
        # Create dataframe of results - used for output
        self.final_results = pd.DataFrame(values, index = row_names, columns = column_names)
        with open(f"simple_{self.sigmad}.tex",'w') as f:
            f.write(self.final_results.to_latex(index=True))


##############################################################################################################################
######################################## Define required Functions ###########################################################
##############################################################################################################################

# Function to solve the First Order Conditions
def equilibrium(eqprice, x, si, mc, beta0, betax, alpha, sigmad):
    num = np.exp(beta0 + betax * x + sigmad * si - alpha * eqprice)
    den = 1 + np.exp(beta0 + betax * x[0] + sigmad * si[0] - alpha * eqprice[0]) + np.exp(beta0 + betax * x[1] + sigmad * si[1] - alpha * eqprice[1])
    foc1 = eqprice[0] - mc[0] - (1 / (alpha * (1 - num[0]/den)))
    foc2 = eqprice[1] - mc[1] - (1 / (alpha * (1 - num[1]/den)))
    return [foc1, foc2]

########################################################################
# Function to generate data and run regressions
def data_reg(m,j, gamma0, gammax, sigmac, gammaw, sigmaomega, beta0, betax, sigmad, alpha):
    # generate required variables
    x =  np.random.randn(m,j)# observed characters of j firms across m markets
    si = np.random.randn(m,j) # unobserved characters of the j firms across m markets
    w = np.random.randn(m,j) # cost shifter observed
    omeg = np.random.randn(m,j) # cost shifter unobserved
    mc = np.exp(gamma0 + gammax * x + sigmac * si + gammaw * w + sigmaomega * omeg) # marginal costs

    # initial guess for price
    initial_price = np.array((mc))
    # place holder for equilibrium price
    price = np.zeros((m,j))

    # Solve for equilibrium for each market:
    for market in range(m):
        price[market] = fsolve(equilibrium, initial_price[market], args = (x[market], si[market], mc[market], beta0, betax, alpha, sigmad))

    # compute demands and shares
    share = np.exp(beta0 + betax * x + sigmad * si - alpha * price)/(1 + np.sum((np.exp(beta0 + betax * x + sigmad * si - alpha * price)),axis = 1).reshape(m, 1))
    shareout = 1 / (1 + np.sum((np.exp(beta0 + betax * x + sigmad * si - alpha * price)),axis = 1).reshape(m, 1)) # outside option
    demand = np.log(share) - np.log(shareout) # dependent variable

    # create data set
    dataj = np.concatenate((np.ones((m, 1)), np.full((m, 1), 2))) #firm index
    datam = np.concatenate((np.arange(1, m+1).reshape(-1, 1), np.arange(1, m+1).reshape(-1, 1))) # market index
    datax = np.vstack(x.T.flatten().reshape(-1,1)) # observed characteristics
    dataxoth = np.vstack((x[:, 1], x[:, 0])).flatten().reshape(-1, 1) # character of other firms
    datap = np.vstack(price.T.flatten().reshape(-1,1)) #equilibrium prices
    datademand = np.vstack(demand.T.flatten().reshape(-1,1)) # market demand
    dataw = np.vstack(w.T.flatten().reshape(-1,1)) # cost shifter
    datamerg = np.column_stack((datam, dataj, datax, datap, datademand, dataw, dataxoth)) # merge data
    finaldf = pd.DataFrame(datamerg, columns = ['M', 'J', 'X', 'P', 'D', 'W', 'XO']) # create dataframe
    
    # Run OLS Regressions
    result_ols = sm.OLS(finaldf['D'], sm.add_constant(finaldf[['P', 'X']])).fit(cov_type='HC3')

    # extract required coefficients
    const_ols = result_ols.params['const']
    betax_ols = result_ols.params['X']
    alpha_ols = result_ols.params['P']

    # extract required standard errors
    se_cosnt_ols = result_ols.get_robustcov_results('HC3').bse[0]
    se_betax_ols = result_ols.get_robustcov_results('HC3').bse[2]
    se_alpha_ols = result_ols.get_robustcov_results('HC3').bse[1]

    # Run IV Regression
    result_iv = IV2SLS(finaldf['D'], sm.add_constant(finaldf[['X']]), finaldf[['P']], finaldf[['W', 'XO']]).fit()

    # extract required coefficients
    const_iv = result_iv.params['const']
    betax_iv = result_iv.params['X']
    alpha_iv = result_iv.params['P']

    # extract required standard errors
    se_const_iv = result_iv.std_errors['const']
    se_betax_iv = result_iv.std_errors['X']
    se_alpha_iv = result_iv.std_errors['P']

    # return as an array
    return pd.DataFrame([[const_ols, betax_ols, alpha_ols, const_iv, betax_iv, alpha_iv]])


In [ ]:
ml_sigmad1 = berry_ml(m=500, # number of markets
               n = 100, # number of montecarlo simulations
               j = 2, # number of firms
               beta0 = 5, # utility parameter constant
               betax = 2, # utility parameter for observed characters
               alpha = 1, # utility parameter for price
               gamma0 = 1, # parameter for cost (intercept)
               gammax = 0.5, # parameter for cost (observed characters)
               gammaw = 0.25, # parameter for cost
               sigmad = 1, # utility parameter (unobservable characters)
               sigmac = 0.25, # cost parameter (unobservable characters)
               sigmaomega = 0.25) # cost parameter (other unobservables))
ml_sigmad1.simulations()
display(Markdown("Results for $\\sigma_d$= 1: "))
display(Markdown(ml_sigmad1.final_results.to_markdown()))

Results for $\sigma_d$= 1: 

|           | True Value   | OLS                   | IV                    |
|:----------|:-------------|:----------------------|:----------------------|
| $\beta_0$ | $5$          | $3.166$ <br> (0.252)  | $4.979$ <br> (0.231)  |
| $\beta_x$ | $2$          | $1.332$ <br> (0.077)  | $1.993$ <br> (0.084)  |
| $\alpha$  | -$1$         | $-0.635$ <br> (0.052) | $-0.996$ <br> (0.046) |

In [9]:
ml_sigmad3 = berry_ml(m=500, # number of markets
               n = 100, # number of montecarlo simulations
               j = 2, # number of firms
               beta0 = 5, # utility parameter constant
               betax = 2, # utility parameter for observed characters
               alpha = 1, # utility parameter for price
               gamma0 = 1, # parameter for cost (intercept)
               gammax = 0.5, # parameter for cost (observed characters)
               gammaw = 0.25, # parameter for cost
               sigmad = 3, # utility parameter (unobservable characters)
               sigmac = 0.25, # cost parameter (unobservable characters)
               sigmaomega = 0.25) # cost parameter (other unobservables))
ml_sigmad3.simulations()
display(Markdown("Results for $\\sigma_d$= 3: "))
display(Markdown(ml_sigmad3.final_results.to_markdown()))


Results for $\sigma_d$= 3: 

|           | True Value   | OLS                  | IV                    |
|:----------|:-------------|:---------------------|:----------------------|
| $\beta_0$ | $5$          | $-0.778$ <br> (0.5)  | $4.925$ <br> (0.751)  |
| $\beta_x$ | $2$          | $0.038$ <br> (0.131) | $1.975$ <br> (0.255)  |
| $\alpha$  | -$1$         | $0.107$ <br> (0.099) | $-0.986$ <br> (0.144) |